In [3]:
import pandas as pd
import numpy as np

In [7]:
gpt4 = pd.read_csv("gpt4.csv")
gpt4

,PatientID,Age,Gender,BMI,BloodPressure_Systolic,BloodPressure_Diastolic,SugarLevel,Cholesterol,HeartRate,Diagnosis,TreatmentCost,VisitsPerYear,Smoker,ExerciseFreq,MedicationAdherence,AdmissionDate,RecentHospitalization
0,522,73,Female,19.8,120,75,51.0,212.0,77,Healthy,549.44,1,False,5,0.21,2021-08-12,False
1,738,70,Male,25.4,125,81,115.0,158.0,78,Healthy,120.22,1,False,2,0.48,2024-03-14,False
2,741,87,Male,25.8,137,79,104.0,173.0,80,Healthy,314.87,2,False,1,0.25,2022-04-15,False
3,661,84,Male,31.8,144,84,142.0,175.0,56,Diabetes,785.55,3,False,1,0.49,2021-08-13,False
4,412,82,Male,27.6,138,81,117.0,200.0,81,Healthy,428.29,2,False,1,0.83,2016-04-21,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,107,31,Male,17.3,102,71,90.0,125.0,74,Healthy,494.02,1,False,0,0.61,2016-10-17,False
996,271,47,Female,19.6,122,73,97.0,147.0,86,Healthy,749.88,2,True,5,0.34,2021-11-30,False
997,861,74,Female,31.0,126,70,140.0,223.0,79,Diabetes,470.28,1,False,5,NaN,2024-10-09,False
998,436,66,Female,26.7,117,75,92.0,158.0,69,Healthy,271.26,3,True,1,0.89,2018-04-25,False


In [111]:
nan_sayisi = gpt4.isna().sum() 
gpt4["BMI"]=gpt4["BMI"].fillna(gpt4["BMI"].mean())
gpt4["SugarLevel"]=gpt4["SugarLevel"].fillna(gpt4["SugarLevel"].mean())
gpt4["Cholesterol"]=gpt4["Cholesterol"].fillna(gpt4["Cholesterol"].mean())
gpt4["MedicationAdherence"]=gpt4["MedicationAdherence"].fillna(gpt4["MedicationAdherence"].mean())
(nan_sayisi / len(gpt4)*gpt4.shape[1])*100

PatientID                  0.0
Age                        0.0
Gender                     0.0
BMI                        0.0
BloodPressure_Systolic     0.0
BloodPressure_Diastolic    0.0
SugarLevel                 0.0
Cholesterol                0.0
HeartRate                  0.0
Diagnosis                  0.0
TreatmentCost              0.0
VisitsPerYear              0.0
Smoker                     0.0
ExerciseFreq               0.0
MedicationAdherence        0.0
AdmissionDate              0.0
RecentHospitalization      0.0
dtype: float64

In [57]:
Q1_T = gpt4["TreatmentCost"].quantile(0.25)
Q3_T = gpt4["TreatmentCost"].quantile(0.75)
IQR_T = Q3_T-Q1_T
alt_T = Q1_T-1.5*IQR_T
ust_T = Q3_T+1.5*IQR_T

Q1_C = gpt4["Cholesterol"].quantile(0.25)
Q3_C = gpt4["Cholesterol"].quantile(0.75)
IQR_C = Q3_C-Q1_C
alt_C = Q1_C-1.5*IQR_C
ust_C = Q3_C+1.5*IQR_C

outlier_gpt4_T = (gpt4["TreatmentCost"]<alt_T) | (gpt4["TreatmentCost"]>ust_T)
outlier_gpt4_C = (gpt4["Cholesterol"]<alt_C) | (gpt4["Cholesterol"]>ust_C)

(outlier_gpt4_T.sum()+outlier_gpt4_C.sum())/ len(gpt4) * 100

np.float64(4.6)

In [77]:
gpt4[outlier_gpt4_T][["BMI","HeartRate"]].mean()

BMI          26.358824
HeartRate    71.285714
dtype: float64

In [79]:
gpt4[outlier_gpt4_C][["BMI","HeartRate"]].mean()

BMI          28.800000
HeartRate    75.181818
dtype: float64

In [147]:
BMI_min = gpt4["BMI"].min()
BMI_max = gpt4["BMI"].max()

SugarLevel_min = gpt4["SugarLevel"].min()
SugarLevel_max = gpt4["SugarLevel"].max()

Cholesterol_min = gpt4["Cholesterol"].min()
Cholesterol_max = gpt4["Cholesterol"].max()

BloodPressureSystolic_min = gpt4["BloodPressure_Systolic"].min()
BloodPressureSystolic_max = gpt4["BloodPressure_Systolic"].max()

BMI_norm = gpt4["BMI"].apply(lambda x: (x - BMI_min) / (BMI_max - BMI_min))
SugarLevel_norm = gpt4["SugarLevel"].apply(lambda x: (x - SugarLevel_min) / (SugarLevel_max - SugarLevel_min))
Cholesterol_norm = gpt4["Cholesterol"].apply(lambda x: (x - Cholesterol_min) / (Cholesterol_max - Cholesterol_min))
BloodPressure_Systolic_norm = gpt4["BloodPressure_Systolic"].apply(lambda x: (x - BloodPressureSystolic_min) / (BloodPressureSystolic_max - BloodPressureSystolic_min))

gpt4["RiskScore"] = (BMI_norm*0.2 + SugarLevel_norm*0.3 + Cholesterol_norm*0.3 + BloodPressure_Systolic_norm*0.2)

gpt4.groupby("Diagnosis")["RiskScore"].mean().sort_values(ascending=False).head(3)

Diagnosis
Hypertension      0.496103
Diabetes          0.473445
Hyperlipidemia    0.473296
Name: RiskScore, dtype: float64

In [149]:
gpt4["RiskScore"].corr(gpt4["TreatmentCost"])

np.float64(0.1840739261985925)

In [153]:
pd.pivot_table(
    gpt4,
    index="Smoker",
    values=["HeartRate","BMI","Cholesterol","TreatmentCost"],
    aggfunc="mean"
)

,BMI,Cholesterol,HeartRate,TreatmentCost
Smoker,,,,
False,27.001456,197.781086,74.362581,416.834361
True,27.773671,202.114580,75.093333,399.439600


In [161]:
gpt4["ExerciseFreq"].corr(gpt4["SugarLevel"])

np.float64(0.0069928599417293374)

In [191]:
gpt4["HealthIndex"] = gpt4["ExerciseFreq"]*0.4 + (1 - gpt4["Smoker"].astype(int))*0.3 + gpt4["MedicationAdherence"]*0.3
gpt4.groupby("Diagnosis")["HealthIndex"].mean().sort_values(ascending=False).head(5)

Diagnosis
Hypertension      172467.656039
Diabetes          172467.654067
Healthy           172467.600006
Hyperlipidemia    172467.593197
Cardiovascular    172467.415420
Name: HealthIndex, dtype: float64

In [213]:
gpt4["AdmissionDate"] = pd.to_datetime(gpt4["AdmissionDate"])

gpt4.groupby(gpt4["AdmissionDate"].dt.year)["TreatmentCost"].mean()

AdmissionDate
2015    392.344872
2016    358.021146
2017    454.052209
2018    397.915889
2019    490.437609
2020    406.438586
2021    349.937273
2022    426.723619
2023    416.733441
2024    460.623366
Name: TreatmentCost, dtype: float64

In [217]:
gpt4.groupby(gpt4["AdmissionDate"].dt.year)["TreatmentCost"].mean().rolling(window=2).apply(lambda x: (x.iloc[1]-x.iloc[0])/x.iloc[0])

AdmissionDate
2015         NaN
2016   -0.087484
2017    0.268227
2018   -0.123634
2019    0.232516
2020   -0.171274
2021   -0.139016
2022    0.219429
2023   -0.023411
2024    0.105319
Name: TreatmentCost, dtype: float64

In [221]:
(gpt4.groupby(gpt4["AdmissionDate"].dt.year)["TreatmentCost"].mean().max()-gpt4.groupby(gpt4["AdmissionDate"].dt.year)["TreatmentCost"].mean().min())/2

70.25016798418972

In [225]:
gpt4[gpt4["AdmissionDate"].dt.year==2024]["RiskScore"].mean()-gpt4[gpt4["AdmissionDate"].dt.year==2023]["RiskScore"].mean()

np.float64(-0.010067387326463006)

In [237]:
gpt4.groupby(gpt4["AdmissionDate"].dt.year)["RecentHospitalization"].sum()

AdmissionDate
2015    5
2016    8
2017    5
2018    4
2019    5
2020    7
2021    8
2022    8
2023    9
2024    2
Name: RecentHospitalization, dtype: int64

In [241]:
gpt4.groupby(gpt4["AdmissionDate"].dt.year)["RecentHospitalization"].sum().rolling(window=2).apply(lambda x: ((x.iloc[1]-x.iloc[0])/x.iloc[0])*100)

AdmissionDate
2015          NaN
2016    60.000000
2017   -37.500000
2018   -20.000000
2019    25.000000
2020    40.000000
2021    14.285714
2022     0.000000
2023    12.500000
2024   -77.777778
Name: RecentHospitalization, dtype: float64